# Problema de Optimización Reducido

In [0]:
# Import PuLP modeler functions
!pip install pulp
from pulp import *

## Creación del problema
Aca lo que le pedimos que sea un problema de programación lineal, con el objetivo de minimizar algo.
El nombre se llama, el problema whiskas https://pythonhosted.org/PuLP/CaseStudies/a_blending_problem.html

In [0]:
prob = LpProblem("The_Whiskas_Problem",LpMinimize) 
prob

The_Whiskas_Problem:
MINIMIZE
None
VARIABLES

## Creación las variables asociadas al problema
Existen 4 parámetros:
1. Nombre de la variable
2. Valor mínimo
3. Valor máximo
4. tipo de dato, discreto (LpInteger) o continuo, por defecto es LpContinuous

In [0]:
x1=LpVariable("ChickenPercent",0,None,LpInteger)
x2=LpVariable("BeefPercent",0)
print(x1)
print(x2)
print(prob) #aca no tenemos nada porque las variables no fueron añadidas al problema

ChickenPercent
BeefPercent
The_Whiskas_Problem:
MINIMIZE
None
VARIABLES



Ahora se le empiezan a agregar las variables al problema, las mismas se agregaran a la variable prob (variable asoaciada a la creación del problema) con el operador +=. La función objetivo es insertada primero, con una coma en el final de cada comando.

## Se añade la función objetivo


In [0]:
prob += 0.013*x1 + 0.008*x2, "Total Cost of Ingredients per can"
print(prob)

The_Whiskas_Problem:
MINIMIZE
0.008*BeefPercent + 0.013*ChickenPercent + 0.0
VARIABLES
BeefPercent Continuous
0 <= ChickenPercent Integer



## Se añaden las restricciones al problema

In [0]:
# The five constraints are entered
prob += x1 + x2 == 100, "PercentagesSum"
prob += 0.100*x1 + 0.200*x2 >= 8.0, "ProteinRequirement"
prob += 0.080*x1 + 0.100*x2 >= 6.0, "FatRequirement"
prob += 0.001*x1 + 0.005*x2 <= 2.0, "FibreRequirement"
prob += 0.002*x1 + 0.005*x2 <= 0.4, "SaltRequirement"
prob

The_Whiskas_Problem:
MINIMIZE
0.008*BeefPercent + 0.013*ChickenPercent + 0.0
SUBJECT TO
PercentagesSum: BeefPercent + ChickenPercent = 100

ProteinRequirement: 0.2 BeefPercent + 0.1 ChickenPercent >= 8

FatRequirement: 0.1 BeefPercent + 0.08 ChickenPercent >= 6

FibreRequirement: 0.005 BeefPercent + 0.001 ChickenPercent <= 2

SaltRequirement: 0.005 BeefPercent + 0.002 ChickenPercent <= 0.4

VARIABLES
BeefPercent Continuous
0 <= ChickenPercent Integer

## Ejecución del problema


Luego de este paso todos los datos quedaron ingresados, la función writeLP() nos permite copiar la información a un archivo .lp(), el cual puede ser abierto para ver de forma prolija el problema.


In [0]:
# The problem data is written to an .lp file
prob.writeLP("WhiskasModel.lp")

[BeefPercent, ChickenPercent]

El problema de programación lineal es resuelto con el solver que PuLP escoge (podria elegirse otro que no sea el por default como ser: prob.solve(CPLEX()) 

In [0]:
prob.solve() # el 1 indica que está resuelto

1

## Status de la solución

El status de la solución puede ser:
* No resuelto
* No factible
* Indefinido
* Optimo

El status es un número, hay que aplicarle el diccionario LpStatus para convertirlo a texto (por eso esta entre [])

In [0]:
print("Status:", LpStatus[prob.status])

Status: Optimal


Los valores que necesitamos de cada una de las variables para alcanzar el óptimo:

In [0]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

BeefPercent = 66.0
ChickenPercent = 34.0


Con estos valores el costo mínimo es de:

In [0]:
print("Total Cost of Ingredients per can = ", value(prob.objective))

Total Cost of Ingredients per can =  0.97


# Problema Completo

Antes de definir la variable prob, los datos claves son incorporados como diccionarios. esto incluye:
* Lista de Ingredientes
* cost de cada Ingrediente
* Porcentaje de cada uno de los cuatro nutrientes

In [0]:
# Creates a list of the Ingredients
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013, 
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.001}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000}

In [0]:
# Creación del problema, el que va a tener los datos
prob = LpProblem("The_Whiskas_Problem", LpMinimize)

In [0]:
# Se crea un dictionario llamado 'ingredient_vars' conteniendo las variables de referencia, y su mínimo que es 0.
ingredient_vars = LpVariable.dicts("Ingr",Ingredients,0)
ingredient_vars

{'BEEF': Ingr_BEEF,
 'CHICKEN': Ingr_CHICKEN,
 'GEL': Ingr_GEL,
 'MUTTON': Ingr_MUTTON,
 'RICE': Ingr_RICE,
 'WHEAT': Ingr_WHEAT}

Como los costos y ingredient_vars son ahora diccionarios con sus claves de referencia como son los nombres de los ingredientes. Los datos pueden ser extraidos como una lista de comprensión como se muestra. LpSum() añadira los elementos de la lista resultante.

## Se añade la función objetivo

In [0]:
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can"
print(prob)

The_Whiskas_Problem:
MINIMIZE
0.008*Ingr_BEEF + 0.013*Ingr_CHICKEN + 0.001*Ingr_GEL + 0.01*Ingr_MUTTON + 0.002*Ingr_RICE + 0.005*Ingr_WHEAT + 0.0
VARIABLES
Ingr_BEEF Continuous
Ingr_CHICKEN Continuous
Ingr_GEL Continuous
Ingr_MUTTON Continuous
Ingr_RICE Continuous
Ingr_WHEAT Continuous



## Se añade las restricciones

In [0]:
# Añadimos las restricciones de la misma manera que la variable objetivo
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"
print(prob)

The_Whiskas_Problem:
MINIMIZE
0.008*Ingr_BEEF + 0.013*Ingr_CHICKEN + 0.001*Ingr_GEL + 0.01*Ingr_MUTTON + 0.002*Ingr_RICE + 0.005*Ingr_WHEAT + 0.0
SUBJECT TO
PercentagesSum: Ingr_BEEF + Ingr_CHICKEN + Ingr_GEL + Ingr_MUTTON + Ingr_RICE
 + Ingr_WHEAT = 100

ProteinRequirement: 0.2 Ingr_BEEF + 0.1 Ingr_CHICKEN + 0.15 Ingr_MUTTON
 + 0.04 Ingr_WHEAT >= 8

FatRequirement: 0.1 Ingr_BEEF + 0.08 Ingr_CHICKEN + 0.11 Ingr_MUTTON
 + 0.01 Ingr_RICE + 0.01 Ingr_WHEAT >= 6

FibreRequirement: 0.005 Ingr_BEEF + 0.001 Ingr_CHICKEN + 0.003 Ingr_MUTTON
 + 0.1 Ingr_RICE + 0.15 Ingr_WHEAT <= 2

SaltRequirement: 0.005 Ingr_BEEF + 0.002 Ingr_CHICKEN + 0.007 Ingr_MUTTON
 + 0.002 Ingr_RICE + 0.008 Ingr_WHEAT <= 0.4

VARIABLES
Ingr_BEEF Continuous
Ingr_CHICKEN Continuous
Ingr_GEL Continuous
Ingr_MUTTON Continuous
Ingr_RICE Continuous
Ingr_WHEAT Continuous



## Ejecución del problema

In [0]:
prob.writeLP("WhiskasModel.lp")

[Ingr_BEEF, Ingr_CHICKEN, Ingr_GEL, Ingr_MUTTON, Ingr_RICE, Ingr_WHEAT]

In [0]:
prob.solve()

1

## Status de la solución


In [0]:
print("Status:", LpStatus[prob.status])

Status: Optimal


Los valores que necesitamos de cada una de las variables para alcanzar el óptimo:

In [0]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

Ingr_BEEF = 60.0
Ingr_CHICKEN = 0.0
Ingr_GEL = 40.0
Ingr_MUTTON = 0.0
Ingr_RICE = 0.0
Ingr_WHEAT = 0.0


Con estos valores el costo mínimo es de:

In [0]:
print("Total Cost of Ingredients per can = ", value(prob.objective))

Total Cost of Ingredients per can =  0.52


# Caso Prueba (simil real) - 1 cliente

En este ejemplo, mostraremos como funcionaría el modelo de optimización para un universo muy acotado, pero que intenta simular un escenario real. Vamos a suponer que tenemos solamente un **cliente** en el banco y dos **productos**, que son:
*   Seguros de automovil
*   Prestamo

Se van a utilizar 3 **canales**, por lo cual vamos a tener 8 combinaciones de canales (3 de a pares + 3 únicos + todos + ninguno). Los canales son:
* SMS: se enviarán promedio dos SMS por persona, en caso de enviar.
* Mail: Se enviarán un promedio de 4 mail por persona.
* Call center: se hará un promedio de 2 llamadas de 5 minutos cada una.

Tendremos **restricciones** de:
* Costo total. El mismo no puede superar cierto umbral.
* Cantidad de contactos previos por canal en los últimos meses: no queremos interactuar con el cliente mas de una cierta cantidad de veces.
* El producto como mucho se ofrecerá por un conjunto de canales.

Vamos a optimizar una ecuación de valor que viene dada por la consideración de:
* Aumento de rentabilidad asociada al éxito de la campaña.
* Probabilidad de éxito de la campaña para el cliente y conjunto de canales.
* Costo asociado al conjunto de canales usado.

El problema se resuelve utilizando programación lineal entera ya que la decisión para cada elemento de la ecuación es binaria (le ofrezco al cliente el producto a por los canales i,j,k o no).

Las unidades están expresadas en pesos Uruguayos.

La ecuación a maximizar es (cliente:i, producto:j, conjunto canal:k):

$Probabilidad_{i,j,k}*Rentabilidad_{j}*Eleccion_{i,j,k}-Costo_{k}$

Sujeta a las restricciones mencionadas.


In [0]:
# Import PuLP modeler functions
!pip install pulp
from pulp import *
import numpy as np
import re

In [0]:
# Crea la lista de optimizadores
Eleccion = ['ALL_SA', 'CC_MAIL_SA', 'CC_SMS_SA', 'MAIL_SMS_SA', 'CC_SA', 'MAIL_SA', 'SMS_SA', 'NONE_SA',
            'ALL_PR', 'CC_MAIL_PR', 'CC_SMS_PR', 'MAIL_SMS_PR', 'CC_PR', 'MAIL_PR', 'SMS_PR', 'NONE_PR']

#Creo la lista de productos
Productos = ['SEGURO_AUTOMOVIL','PRESTAMO']

#Creo la lista de canales
Canales = ['CC','MAIL','SMS']

#Creo la lista de Rentabilidad Productos
Rentabilidad_Producto = {'SEGURO_AUTOMOVIL':3800,
                         'PRESTAMO':4500}

# Aumento de la rentabilidad asociada a cada producto (podría ser distinto en cada uno de los canales)
Rentabilidad = {'ALL_SA':Rentabilidad_Producto['SEGURO_AUTOMOVIL'], 
                'CC_MAIL_SA':Rentabilidad_Producto['SEGURO_AUTOMOVIL'],
                'CC_SMS_SA':Rentabilidad_Producto['SEGURO_AUTOMOVIL'], 
                'MAIL_SMS_SA':Rentabilidad_Producto['SEGURO_AUTOMOVIL'], 
                'CC_SA':Rentabilidad_Producto['SEGURO_AUTOMOVIL'],
                'MAIL_SA':Rentabilidad_Producto['SEGURO_AUTOMOVIL'], 
                'SMS_SA':Rentabilidad_Producto['SEGURO_AUTOMOVIL'], 
                'NONE_SA':Rentabilidad_Producto['SEGURO_AUTOMOVIL'],
                
                'ALL_PR':Rentabilidad_Producto['PRESTAMO'], 
                'CC_MAIL_PR':Rentabilidad_Producto['PRESTAMO'],
                'CC_SMS_PR':Rentabilidad_Producto['PRESTAMO'], 
                'MAIL_SMS_PR':Rentabilidad_Producto['PRESTAMO'], 
                'CC_PR':Rentabilidad_Producto['PRESTAMO'],
                'MAIL_PR':Rentabilidad_Producto['PRESTAMO'], 
                'SMS_PR':Rentabilidad_Producto['PRESTAMO'], 
                'NONE_PR':Rentabilidad_Producto['PRESTAMO']}

#Probabilidad de éxito
#deberia venir de un modelo de probabilidad de éxito, pero como no lo tenemos la imputamos de forma aleatoria.
Probabilidad = {'NONE_SA':np.random.uniform(0, 0.001, size=None),
               'NONE_PR':np.random.uniform(0, 0.001, size=None)}

Probabilidad.update({'CC_SA': np.random.uniform(Probabilidad['NONE_SA'], 0.1, size=None), 
                    'MAIL_SA':np.random.uniform(Probabilidad['NONE_SA'] , 0.01, size=None),  
                    'SMS_SA':np.random.uniform(Probabilidad['NONE_SA'] , 0.025, size=None), 
                    'CC_PR':np.random.uniform(Probabilidad['NONE_PR'], 0.1, size=None),
                    'MAIL_PR':np.random.uniform(Probabilidad['NONE_PR'] , 0.01, size=None),
                    'SMS_PR':np.random.uniform(Probabilidad['NONE_PR'] , 0.025, size=None)})

Probabilidad.update({'CC_MAIL_SA':np.random.uniform(min([Probabilidad['MAIL_SA'],Probabilidad['CC_SA']]), 0.110, size=None),
                      'CC_SMS_SA': np.random.uniform(min([Probabilidad['SMS_SA'],Probabilidad['CC_SA']]), 0.125, size=None), 
                      'MAIL_SMS_SA':np.random.uniform(min([Probabilidad['MAIL_SA'],Probabilidad['SMS_SA']]), 0.035, size=None), 
                      'CC_MAIL_PR':np.random.uniform(min([Probabilidad['MAIL_PR'],Probabilidad['CC_PR']]), 0.110, size=None),
                      'CC_SMS_PR': np.random.uniform(min([Probabilidad['SMS_PR'],Probabilidad['CC_PR']]), 0.125, size=None), 
                      'MAIL_SMS_PR':np.random.uniform(min([Probabilidad['MAIL_PR'],Probabilidad['SMS_PR']]), 0.035, size=None)})

Probabilidad.update({'ALL_SA':np.random.uniform(min([Probabilidad['CC_MAIL_SA'],Probabilidad['CC_SMS_SA'],Probabilidad['MAIL_SMS_SA']]), 0.135, size=None),
                     'ALL_PR':np.random.uniform(min([Probabilidad['CC_MAIL_PR'],Probabilidad['CC_SMS_PR'],Probabilidad['MAIL_SMS_PR']]), 0.135, size=None)})


#Costos por canal
Costos_canal={'CC': 50, 'MAIL': 0.40, 'SMS':2}

#restricción de costo
Costo_total=53

#Cantidad de interacciones últimos 3 meses
Usos_canal={'CC': np.random.binomial(n=2, p=0.4, size=1), 
              'MAIL': np.random.binomial(n=8, p=0.9, size=1), 
              'SMS':np.random.binomial(n=4, p=0.7, size=1),
              'NONE':0} 
Usos_canal.update({'ALL':Usos_canal['CC']+Usos_canal['MAIL']+Usos_canal['SMS']})


#Restricción por interacciones últimos 3 meses
Restriccion_canal={'CC': 2, 
                  'MAIL': 8, 
                  'SMS':4,
                  'NONE':1}
Restriccion_canal.update({'ALL':Restriccion_canal['CC']+Restriccion_canal['MAIL']+Restriccion_canal['SMS']})


#Crea la lista de costos
Costos = {'ALL_SA':Costos_canal['CC']+Costos_canal['MAIL']+Costos_canal['SMS'], 
          'CC_MAIL_SA':Costos_canal['CC']+Costos_canal['MAIL'],
          'CC_SMS_SA':Costos_canal['CC']+Costos_canal['SMS'], 
          'MAIL_SMS_SA':Costos_canal['MAIL']+Costos_canal['SMS'], 
          'CC_SA':Costos_canal['CC'],
          'MAIL_SA':Costos_canal['MAIL'], 
          'SMS_SA':Costos_canal['SMS'], 
          'NONE_SA':0,

          'ALL_PR':Costos_canal['CC']+Costos_canal['MAIL']+Costos_canal['SMS'], 
          'CC_MAIL_PR':Costos_canal['CC']+Costos_canal['MAIL'],
          'CC_SMS_PR':Costos_canal['CC']+Costos_canal['SMS'], 
          'MAIL_SMS_PR':Costos_canal['MAIL']+Costos_canal['SMS'], 
          'CC_PR':Costos_canal['CC'],
          'MAIL_PR':Costos_canal['MAIL'], 
          'SMS_PR':Costos_canal['SMS'], 
          'NONE_PR':0}

Se crean las variables, especificando que van a ser enteros y van a valer 0 o 1.

In [0]:
Eleccion_vars = LpVariable.dicts("E",Eleccion,0,1,LpInteger)
Eleccion_vars

{'ALL_PR': E_ALL_PR,
 'ALL_SA': E_ALL_SA,
 'CC_MAIL_PR': E_CC_MAIL_PR,
 'CC_MAIL_SA': E_CC_MAIL_SA,
 'CC_PR': E_CC_PR,
 'CC_SA': E_CC_SA,
 'CC_SMS_PR': E_CC_SMS_PR,
 'CC_SMS_SA': E_CC_SMS_SA,
 'MAIL_PR': E_MAIL_PR,
 'MAIL_SA': E_MAIL_SA,
 'MAIL_SMS_PR': E_MAIL_SMS_PR,
 'MAIL_SMS_SA': E_MAIL_SMS_SA,
 'NONE_PR': E_NONE_PR,
 'NONE_SA': E_NONE_SA,
 'SMS_PR': E_SMS_PR,
 'SMS_SA': E_SMS_SA}

In [0]:
#me quedo con las variables inherentes al prestamo
Eleccion_PR=[]
for i in list(Eleccion_vars.keys()):
  match=re.search(r'PR',i)
  if match:
    Eleccion_PR.append(i)

#me quedo con las variables inherentes al Seguro Automovil
Eleccion_SA=[]
for i in list(Eleccion_vars.keys()):
  match=re.search(r'SA', i)
  if match:
    Eleccion_SA.append(i)

In [0]:
#creo diccionario de póliticas por canal
contact_policy={}
for j in Canales:
  for i in list(Eleccion_vars.keys()):
    match=re.search(j, i)
    if (match) and (Usos_canal[j][0]==Restriccion_canal[j]):
      contact_policy.update({i:0})
    elif (match) and (Usos_canal[j][0]!=Restriccion_canal[j]):
      contact_policy.update({i:1})

In [0]:
# Creación del problema, el que va a tener los datos
prob = LpProblem("The_Marketing_Problem", LpMaximize)

### Función objetivo

In [0]:
prob += lpSum([(Probabilidad[i]*Rentabilidad[i]-Costos[i])*Eleccion_vars[i] for i in Eleccion]), "Total Rentabilidad"
print(prob)

The_Marketing_Problem:
MAXIMIZE
274.57978710672444*E_ALL_PR + 108.95241580243948*E_ALL_SA + 322.0494327024444*E_CC_MAIL_PR + 306.34435806175685*E_CC_MAIL_SA + -28.418383092564703*E_CC_PR + 54.54105334543763*E_CC_SA + 50.945923876879675*E_CC_SMS_PR + 343.1562714125423*E_CC_SMS_SA + 14.325513025760115*E_MAIL_PR + 32.89727505676687*E_MAIL_SA + 107.23729480933945*E_MAIL_SMS_PR + 75.29687607178529*E_MAIL_SMS_SA + 1.5410243859549928*E_NONE_PR + 3.203511018172277*E_NONE_SA + 64.85344768065656*E_SMS_PR + 31.996873406054753*E_SMS_SA + 0.0
VARIABLES
0 <= E_ALL_PR <= 1 Integer
0 <= E_ALL_SA <= 1 Integer
0 <= E_CC_MAIL_PR <= 1 Integer
0 <= E_CC_MAIL_SA <= 1 Integer
0 <= E_CC_PR <= 1 Integer
0 <= E_CC_SA <= 1 Integer
0 <= E_CC_SMS_PR <= 1 Integer
0 <= E_CC_SMS_SA <= 1 Integer
0 <= E_MAIL_PR <= 1 Integer
0 <= E_MAIL_SA <= 1 Integer
0 <= E_MAIL_SMS_PR <= 1 Integer
0 <= E_MAIL_SMS_SA <= 1 Integer
0 <= E_NONE_PR <= 1 Integer
0 <= E_NONE_SA <= 1 Integer
0 <= E_SMS_PR <= 1 Integer
0 <= E_SMS_SA <= 1 Inte

### Restricciones


In [0]:
# Añadimos las restricciones de la misma manera que la variable objetivo
prob += lpSum([Eleccion_vars[i] for i in Eleccion_PR]) == 1, "EleccionFinal_PR" #haya alguna acción para el producto préstamo
prob += lpSum([Eleccion_vars[i] for i in Eleccion_SA]) == 1, "EleccionFinal_SA" #haya alguna acción para el producto Seguro automovil

# restricciones de contact policy
for i in list(contact_policy.keys()):
  prob += Eleccion_vars[i]<= contact_policy[i] , "ContactPolicy %s" %  str(i)

# restricción de costo
prob += lpSum([Costos[i]*Eleccion_vars[i] for i in Eleccion]) <= Costo_total, "Costo total" 


### Ejecución del problema, status de la solución y acciones

In [0]:
#prob.writeLP("OptimizationProblem.lp")

In [0]:
prob.solve()

1

In [0]:
print("Status:", LpStatus[prob.status])

Status: Optimal


Imprimo el conjunto de acciones a realizar

In [0]:
print('Acciones:')
for v in prob.variables():
  if (v.varValue==1):
  #Canal
    if re.search('NONE', v.name):
      V1='NO ofrecer'
    elif re.search('CC_ALL', v.name):
      V1='Ofrecer por TODOS los canales'
    elif re.search('CC_SMS', v.name):
      V1='Ofrecer por SMS y CALL CENTER'
    elif re.search('CC_MAIL', v.name):
      V1='Ofrecer por MAIL y CALL CENTER'
    elif re.search('MAIL_SMS', v.name):
      V1='Ofrecer por MAIL y SMS'
    elif re.search('CC', v.name):
      V1='Ofrecer por el CALL CENTER'
    elif re.search('SMS', v.name):
      V1='Ofrecer por SMS'
    elif re.search('MAIL', v.name):
      V1='Ofrecer por MAIL'
    else:
      V1='Ofrecer por TODOS los canales'
  #producto
    if re.search('SA', v.name):
      V2='Seguro de Automovil'
    else:
      V2='Prestamo'
    print(V1+' el producto '+V2+' al cliente')

Acciones:
Ofrecer por TODOS los canales el producto Prestamo al cliente
NO ofrecer el producto Seguro de Automovil al cliente


In [0]:
print("Función de valor = ", value(prob.objective))

Función de valor =  277.7832981248967


# Caso Prueba (simil real) - 10 clientes

En este ejemplo, mostraremos como funcionaría el modelo de optimización para un universo muy acotado, pero que intenta simular un escenario real. Vamos a suponer que tenemos solamente 10 **clientes** en el banco y dos **productos**, que son:
*   Seguros de automovil
*   Prestamo

Se van a utilizar 3 **canales**, por lo cual vamos a tener 8 combinaciones de canales (3 de a pares + 3 únicos + todos + ninguno). Los canales son:
* SMS: se enviarán promedio dos SMS por persona, en caso de enviar.
* Mail: Se enviarán un promedio de 4 mail por persona.
* Call center: se hará un promedio de 2 llamadas de 5 minutos cada una.

Tendremos **restricciones** de:
* Costo total. El mismo no puede superar cierto umbral por presupesto de marketing.
* costo por cliente.
* el cliente no quiere ser contactado por algún canal
* Cantidad de contactos previos por canal en los últimos meses: no queremos interactuar con el cliente mas de una cierta cantidad de veces.
* El producto como mucho se ofrecerá por un conjunto de canales.
* capacidad del call center no puede superar un cierto umbral

Vamos a optimizar una ecuación de valor que viene dada por la consideración de:
* Aumento de rentabilidad asociada al éxito de la campaña.
* Probabilidad de éxito de la campaña para el cliente y conjunto de canales.
* Costo asociado al conjunto de canales usado.

El problema se resuelve utilizando programación lineal entera ya que la decisión para cada elemento de la ecuación es binaria (le ofrezco al cliente el producto a por los canales i,j,k o no).

Las unidades están expresadas en pesos Uruguayos.

La ecuación a maximizar es (cliente: i, producto: j, conjunto canal: k):

$Probabilidad_{i,j,k}*Rentabilidad_{j}*Eleccion_{i,j,k}-Costo_{k}$

Sujeta a las restricciones mencionadas.


In [1]:
# Import PuLP modeler functions
!pip install pulp
from pulp import *
import numpy as np
import pandas as pd
import re

     |████████████████████████████████| 39.2MB 102kB/s 


In [0]:
#Creo la lista de productos
Productos = ['SEGURO_AUTOMOVIL','PRESTAMO']

#Creo la lista de canales
Canales = ['CC','MAIL','SMS']

#Elecciones posibles
El = ['ALL_SA', 'CC_MAIL_SA', 'CC_SMS_SA', 'MAIL_SMS_SA', 'CC_SA', 'MAIL_SA', 'SMS_SA', 'NONE_SA',
            'ALL_PR', 'CC_MAIL_PR', 'CC_SMS_PR', 'MAIL_SMS_PR', 'CC_PR', 'MAIL_PR', 'SMS_PR', 'NONE_PR']

#Costos por canal
Costos_canal=dict(zip(Canales,[50, 0.40, 2]))

#restricción de costo (presupuesto Marketing)
Costo_total=500

#restricción de costo por cliente
Costo_cliente=100

#restricción  capacidad call center
Capacidad=6

#Restricción por interacciones últimos 3 meses
Restriccion_canal={'CC': 2, 
                  'MAIL': 8, 
                  'SMS':4,
                  'NONE':1}

Restriccion_canal.update({'ALL':Restriccion_canal['CC']+Restriccion_canal['MAIL']+Restriccion_canal['SMS']})

In [0]:
Eleccion=[]
Rentabilidad_Producto = {}
Rentabilidad={}
Costos={}
Probabilidad={}

#elección del cliente
for id in range(10):
  cliente='_Cl'+str(id)


  #rentabilidad por producto
  Rentabilidad_Producto.update({Productos[0]+cliente: np.random.gamma(shape=12000, scale=0.25),
                          Productos[1]+cliente:np.random.gamma(shape=16000, scale=0.25)})


  #Cantidad de interacciones últimos 3 meses
  Usos_canal={'CC'+cliente: np.random.binomial(n=2, p=0.4, size=None), 
              'MAIL'+cliente: np.random.binomial(n=8, p=0.9, size=None), 
              'SMS'+cliente:np.random.binomial(n=4, p=0.7, size=None),
              'NONE'+cliente:0} 
  Usos_canal.update({'ALL'+cliente:Usos_canal['CC'+cliente]+Usos_canal['MAIL'+cliente]+Usos_canal['SMS'+cliente]})


  for i in range(len(El)):
    Eleccion.append(El[i]+cliente)
    if re.search('SA', El[i]):
        Rentabilidad.update({El[i]+cliente:Rentabilidad_Producto[Productos[0]+cliente]})
    else:
        Rentabilidad.update({El[i]+cliente:Rentabilidad_Producto[Productos[1]+cliente]})


  #Probabilidad de éxito
  #deberia venir de un modelo de probabilidad de éxito, pero como no lo tenemos la imputamos de forma aleatoria.
  Probabilidad.update({'NONE_SA'+cliente:np.random.uniform(0, 0.001, size=None),
                       'NONE_PR'+cliente:np.random.uniform(0, 0.001, size=None)})

  Probabilidad.update({'CC_SA'+cliente: np.random.uniform(Probabilidad['NONE_SA'+cliente], 0.1, size=None), 
                      'MAIL_SA'+cliente:np.random.uniform(Probabilidad['NONE_SA'+cliente] , 0.01, size=None),  
                      'SMS_SA'+cliente:np.random.uniform(Probabilidad['NONE_SA'+cliente] , 0.025, size=None), 
                      'CC_PR'+cliente:np.random.uniform(Probabilidad['NONE_PR'+cliente], 0.1, size=None),
                      'MAIL_PR'+cliente:np.random.uniform(Probabilidad['NONE_PR'+cliente] , 0.01, size=None),
                      'SMS_PR'+cliente:np.random.uniform(Probabilidad['NONE_PR'+cliente] , 0.025, size=None)})

  Probabilidad.update({'CC_MAIL_SA'+cliente:np.random.uniform(min([Probabilidad['MAIL_SA'+cliente],Probabilidad['CC_SA'+cliente]]), 0.110, size=None),
                        'CC_SMS_SA'+cliente: np.random.uniform(min([Probabilidad['SMS_SA'+cliente],Probabilidad['CC_SA'+cliente]]), 0.125, size=None), 
                        'MAIL_SMS_SA'+cliente:np.random.uniform(min([Probabilidad['MAIL_SA'+cliente],Probabilidad['SMS_SA'+cliente]]), 0.035, size=None), 
                        'CC_MAIL_PR'+cliente:np.random.uniform(min([Probabilidad['MAIL_PR'+cliente],Probabilidad['CC_PR'+cliente]]), 0.110, size=None),
                        'CC_SMS_PR'+cliente: np.random.uniform(min([Probabilidad['SMS_PR'+cliente],Probabilidad['CC_PR'+cliente]]), 0.125, size=None), 
                        'MAIL_SMS_PR'+cliente:np.random.uniform(min([Probabilidad['MAIL_PR'+cliente],Probabilidad['SMS_PR'+cliente]]), 0.035, size=None)})

  Probabilidad.update({'ALL_SA'+cliente:np.random.uniform(min([Probabilidad['CC_MAIL_SA'+cliente],Probabilidad['CC_SMS_SA'+cliente],Probabilidad['MAIL_SMS_SA'+cliente]]), 0.135, size=None),
                      'ALL_PR'+cliente:np.random.uniform(min([Probabilidad['CC_MAIL_PR'+cliente],Probabilidad['CC_SMS_PR'+cliente],Probabilidad['MAIL_SMS_PR'+cliente]]), 0.135, size=None)})


  #Crea la lista de costos
  Costos.update({'ALL_SA'+cliente:Costos_canal['CC']+Costos_canal['MAIL']+Costos_canal['SMS'], 
                'CC_MAIL_SA'+cliente:Costos_canal['CC']+Costos_canal['MAIL'],
                'CC_SMS_SA'+cliente:Costos_canal['CC']+Costos_canal['SMS'], 
                'MAIL_SMS_SA'+cliente:Costos_canal['MAIL']+Costos_canal['SMS'], 
                'CC_SA'+cliente:Costos_canal['CC'],
                'MAIL_SA'+cliente:Costos_canal['MAIL'], 
                'SMS_SA'+cliente:Costos_canal['SMS'], 
                'NONE_SA'+cliente:0,

                'ALL_PR'+cliente:Costos_canal['CC']+Costos_canal['MAIL']+Costos_canal['SMS'], 
                'CC_MAIL_PR'+cliente:Costos_canal['CC']+Costos_canal['MAIL'],
                'CC_SMS_PR'+cliente:Costos_canal['CC']+Costos_canal['SMS'], 
                'MAIL_SMS_PR'+cliente:Costos_canal['MAIL']+Costos_canal['SMS'], 
                'CC_PR'+cliente:Costos_canal['CC'],
                'MAIL_PR'+cliente:Costos_canal['MAIL'], 
                'SMS_PR'+cliente:Costos_canal['SMS'], 
                'NONE_PR'+cliente:0})



  #creo diccionario de póliticas por canal
  contact_policy={}
  for j in Canales:
    for i in list(Probabilidad.keys()):
      match=re.search(j, i)
      if (match) and (Usos_canal[j+cliente]==Restriccion_canal[j]):
        contact_policy.update({i:0})
      elif (match) and (Usos_canal[j+cliente]!=Restriccion_canal[j]):
        contact_policy.update({i:np.random.binomial(n=1, p=0.95, size=None)}) #aca le doy un 5% de chances que el cliente diga que no quiere el producto


In [0]:
# Creación del problema, el que va a tener los datos
prob = LpProblem("The_Marketing_Problem", LpMaximize)

In [0]:
Eleccion_vars = LpVariable.dicts("E",Eleccion,0,1,LpInteger)
#Eleccion_vars

### Función objetivo

In [0]:
prob += lpSum([(Probabilidad[i]*Rentabilidad[i]-Costos[i])*Eleccion_vars[i] for i in Eleccion]), "Total Rentabilidad"
#print(prob)

### Restricciones a nivel cliente


In [0]:
for id in range(10):
  cliente='_Cl'+str(id)
  #me quedo con las variables inherentes al prestamo
  Eleccion_PR=[]
  for i in list(Probabilidad.keys()):
    match=re.search(r'PR'+cliente,i)
    if match:
      Eleccion_PR.append(i)

  #me quedo con las variables inherentes al Seguro Automovil
  Eleccion_SA=[]
  for i in list(Probabilidad.keys()):
    match=re.search(r'SA'+cliente, i)
    if match:
      Eleccion_SA.append(i)

  #me quedo con las variables inherentes al cliente
  Eleccion_Cl=[]
  for i in list(Probabilidad.keys()):
    match=re.search(cliente, i)
    if match:
      Eleccion_Cl.append(i)

  # Añadimos las restricciones de la misma manera que la variable objetivo
  prob += lpSum([Eleccion_vars[i] for i in Eleccion_PR]) == 1, "EleccionFinal_PR %s" %  str(cliente) #haya alguna acción para el producto préstamo
  prob += lpSum([Eleccion_vars[i] for i in Eleccion_SA]) == 1, "EleccionFinal_SA %s" %  str(cliente) #haya alguna acción para el producto Seguro automovil
  prob += lpSum([Costos[i]*Eleccion_vars[i] for i in Eleccion_Cl]) <= Costo_cliente, "costo %s" %  str(cliente) #haya alguna acción para el producto Seguro automovil

In [0]:
#Restricciones genereales
# restricciones de contact policy
for i in list(contact_policy.keys()):
  prob += Eleccion_vars[i]<= contact_policy[i] , "ContactPolicy %s" %  str(i)

# restricción de costo
prob += lpSum([Costos[i]*Eleccion_vars[i] for i in Eleccion]) <= Costo_total, "Costo total" 

#me quedo con las variables inherentes a call center
Eleccion_CC=[]
for i in list(Probabilidad.keys()):
  match_cc=re.search(r'CC', i)
  match_all=re.search(r'ALL', i)
  if (match_cc or match_all):
    Eleccion_CC.append(i)
# restricción de capacidad
prob += lpSum([Eleccion_vars[i] for i in Eleccion_CC]) <= Capacidad, "Capacidad Call Center" 

### Ejecución del problema, status de la solución y acciones

In [0]:
prob.writeLP("The_Marketing_Problem.lp")

[E_ALL_PR_Cl0,
 E_ALL_PR_Cl1,
 E_ALL_PR_Cl2,
 E_ALL_PR_Cl3,
 E_ALL_PR_Cl4,
 E_ALL_PR_Cl5,
 E_ALL_PR_Cl6,
 E_ALL_PR_Cl7,
 E_ALL_PR_Cl8,
 E_ALL_PR_Cl9,
 E_ALL_SA_Cl0,
 E_ALL_SA_Cl1,
 E_ALL_SA_Cl2,
 E_ALL_SA_Cl3,
 E_ALL_SA_Cl4,
 E_ALL_SA_Cl5,
 E_ALL_SA_Cl6,
 E_ALL_SA_Cl7,
 E_ALL_SA_Cl8,
 E_ALL_SA_Cl9,
 E_CC_MAIL_PR_Cl0,
 E_CC_MAIL_PR_Cl1,
 E_CC_MAIL_PR_Cl2,
 E_CC_MAIL_PR_Cl3,
 E_CC_MAIL_PR_Cl4,
 E_CC_MAIL_PR_Cl5,
 E_CC_MAIL_PR_Cl6,
 E_CC_MAIL_PR_Cl7,
 E_CC_MAIL_PR_Cl8,
 E_CC_MAIL_PR_Cl9,
 E_CC_MAIL_SA_Cl0,
 E_CC_MAIL_SA_Cl1,
 E_CC_MAIL_SA_Cl2,
 E_CC_MAIL_SA_Cl3,
 E_CC_MAIL_SA_Cl4,
 E_CC_MAIL_SA_Cl5,
 E_CC_MAIL_SA_Cl6,
 E_CC_MAIL_SA_Cl7,
 E_CC_MAIL_SA_Cl8,
 E_CC_MAIL_SA_Cl9,
 E_CC_PR_Cl0,
 E_CC_PR_Cl1,
 E_CC_PR_Cl2,
 E_CC_PR_Cl3,
 E_CC_PR_Cl4,
 E_CC_PR_Cl5,
 E_CC_PR_Cl6,
 E_CC_PR_Cl7,
 E_CC_PR_Cl8,
 E_CC_PR_Cl9,
 E_CC_SA_Cl0,
 E_CC_SA_Cl1,
 E_CC_SA_Cl2,
 E_CC_SA_Cl3,
 E_CC_SA_Cl4,
 E_CC_SA_Cl5,
 E_CC_SA_Cl6,
 E_CC_SA_Cl7,
 E_CC_SA_Cl8,
 E_CC_SA_Cl9,
 E_CC_SMS_PR_Cl0,
 E_CC_SMS_PR_Cl1,
 E_C

In [0]:
prob.solve()

1

In [0]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [0]:
print('Acciones:')
for v in prob.variables():
  if (v.varValue==1):
  #Canal
    if re.search('NONE', v.name):
      V1='NO ofrecer'
    elif re.search('CC_ALL', v.name):
      V1='Ofrecer por TODOS los canales'
    elif re.search('CC_SMS', v.name):
      V1='Ofrecer por SMS y CALL CENTER'
    elif re.search('CC_MAIL', v.name):
      V1='Ofrecer por MAIL y CALL CENTER'
    elif re.search('MAIL_SMS', v.name):
      V1='Ofrecer por MAIL y SMS'
    elif re.search('CC', v.name):
      V1='Ofrecer por el CALL CENTER'
    elif re.search('SMS', v.name):
      V1='Ofrecer por SMS'
    elif re.search('MAIL', v.name):
      V1='Ofrecer por MAIL'
    else:
      V1='Ofrecer por TODOS los canales'
  #producto
    if re.search('SA', v.name):
      V2='Seguro de Automovil'
    else:
      V2='Prestamo'
    print(V1+' el producto '+V2+' al cliente '+v.name[-1:])

Acciones:
Ofrecer por TODOS los canales el producto Seguro de Automovil al cliente 0
Ofrecer por TODOS los canales el producto Seguro de Automovil al cliente 1
Ofrecer por TODOS los canales el producto Seguro de Automovil al cliente 3
Ofrecer por TODOS los canales el producto Seguro de Automovil al cliente 7
Ofrecer por SMS y CALL CENTER el producto Prestamo al cliente 5
Ofrecer por SMS y CALL CENTER el producto Seguro de Automovil al cliente 2
Ofrecer por MAIL y SMS el producto Prestamo al cliente 0
Ofrecer por MAIL y SMS el producto Prestamo al cliente 7
Ofrecer por MAIL y SMS el producto Prestamo al cliente 8
Ofrecer por MAIL y SMS el producto Prestamo al cliente 9
Ofrecer por MAIL y SMS el producto Seguro de Automovil al cliente 4
Ofrecer por MAIL y SMS el producto Seguro de Automovil al cliente 6
Ofrecer por MAIL y SMS el producto Seguro de Automovil al cliente 9
Ofrecer por SMS el producto Prestamo al cliente 1
Ofrecer por SMS el producto Prestamo al cliente 2
Ofrecer por SMS el 

In [0]:
print("Función de valor = ", value(prob.objective))

Función de valor =  3198.3553333548875
